### Importing Libraries
This section imports the necessary libraries for the project:
- **os**: Used for file and directory manipulations.
- **cv2**: OpenCV library for image processing tasks.
- **xml.etree.ElementTree**: For parsing XML files, useful for reading annotations.
- **shutil**: Provides functions for high-level file operations such as copying files.
- **random**: Used for generating random numbers, which can help in shuffling datasets.
- **matplotlib.pyplot**: A library for visualizing data and images.

In [12]:
import os
import cv2
import xml.etree.ElementTree as ET
import shutil
import random
import matplotlib.pyplot as plt

### Organizing Image and XML Files
This code snippet organizes image and XML annotation files into separate directories:
- **Base Directory**: Specifies the path where the original images and XML files are stored.
- **Output Directories**: Creates two directories, `jpeg_files` and `xml_files`, to store the copied files.
- **File Iteration**: Loops through all files in the base directory:
  - Checks for files ending with `.jpeg` and copies them to the `jpeg_files` directory.
  - Checks for files ending with `.xml` and copies them to the `xml_files` directory.
- **Completion Message**: Prints a message indicating that the files have been successfully copied into their respective folders.

In [ ]:
base_dir = '/content/drive/MyDrive/images'  

jpeg_dir = '/content/drive/MyDrive/jpeg_files'
xml_dir = '/content/drive/MyDrive/xml_files'

os.makedirs(jpeg_dir, exist_ok=True)
os.makedirs(xml_dir, exist_ok=True)

for filename in os.listdir(base_dir):
    if filename.endswith('.jpeg'):
        shutil.copy(os.path.join(base_dir, filename), os.path.join(jpeg_dir, filename))
    elif filename.endswith('.xml'):
        shutil.copy(os.path.join(base_dir, filename), os.path.join(xml_dir, filename))

print("Files have been copied into 'jpeg_files' and 'xml_files'.")

### Counting JPEG and XML Files
This code snippet counts and displays the number of JPEG and XML files in their respective directories:
- **JPEG Count**: Uses a list comprehension to iterate through the `jpeg_dir`, counting files that end with `.jpeg`.
- **XML Count**: Similarly counts files in the `xml_dir` that end with `.xml`.
- **Output**: Prints the total counts of both JPEG and XML files, providing a quick overview of the dataset's structure.

In [ ]:
jpeg_count = len([filename for filename in os.listdir(jpeg_dir) if filename.endswith('.jpeg')])

xml_count = len([filename for filename in os.listdir(xml_dir) if filename.endswith('.xml')])

print(f"Number of JPEG files: {jpeg_count}")
print(f"Number of XML files: {xml_count}")

Number of JPEG files: 228
Number of XML files: 225


### Preparing the Dataset for YOLO
This code snippet prepares the dataset for training a YOLO model by organizing image and annotation files:
- **Directory Setup**: Defines paths for JPEG and XML files and creates a structured dataset directory containing separate folders for training and validation images and labels.
- **Function `convert_voc_to_yolo`**: Converts bounding box annotations from PASCAL VOC format (XML) to YOLO format. This involves calculating the normalized coordinates for the center of the bounding box and its dimensions relative to the image size.
- **File Collection and Shuffling**: Collects JPEG files from the specified directory and shuffles them for randomness.
- **Dataset Splitting**: Splits the shuffled image list into training (80%) and validation (20%) sets.
- **Copying Images and Generating Labels**: Iterates through the training and validation files:
  - Copies images to their respective directories.
  - Checks for the existence of corresponding XML files and converts them to YOLO format labels, saving them as `.txt` files.
- **Completion Message**: Prints a confirmation that the dataset structure has been successfully created.


In [ ]:
jpeg_dir = '/content/drive/MyDrive/jpeg_files'
xml_dir = '/content/drive/MyDrive/xml_files'
dataset_dir = '/content/drive/MyDrive/dataset'

train_images_dir = os.path.join(dataset_dir, 'images', 'train')
train_labels_dir = os.path.join(dataset_dir, 'labels', 'train')
val_images_dir = os.path.join(dataset_dir, 'images', 'val')
val_labels_dir = os.path.join(dataset_dir, 'labels', 'val')

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

def convert_voc_to_yolo(xml_file, img_width, img_height):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    yolo_data = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        class_id = 0  

        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)

        # Convert to YOLO format
        x_center = (xmin + xmax) / 2 / img_width
        y_center = (ymin + ymax) / 2 / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height

        yolo_data.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return yolo_data

image_files = []

for filename in os.listdir(jpeg_dir):
    if filename.endswith('.jpeg'):
        image_files.append(filename)

random.shuffle(image_files)

split_index = int(len(image_files) * 0.8)
train_files = image_files[:split_index]
val_files = image_files[split_index:]

for filename in train_files + val_files:
    image_path = os.path.join(jpeg_dir, filename)

    if filename in train_files:
        target_image_dir = train_images_dir
        target_label_dir = train_labels_dir
    else:
        target_image_dir = val_images_dir
        target_label_dir = val_labels_dir

    shutil.copy(image_path, os.path.join(target_image_dir, filename))

    xml_file_path = os.path.join(xml_dir, os.path.splitext(filename)[0] + '.xml')

    if os.path.exists(xml_file_path):
        img = cv2.imread(image_path)
        img_height, img_width, _ = img.shape

        yolo_labels = convert_voc_to_yolo(xml_file_path, img_width, img_height)
        label_file_path = os.path.join(target_label_dir, f"{os.path.splitext(filename)[0]}.txt")

        with open(label_file_path, 'w') as f:
            for line in yolo_labels:
                f.write(line + '\n')
    else:
        print(f"No XML file found for {filename}, skipping label generation.")

print("Dataset structure created with train and validation sets.")

No XML file found for N182.jpeg, skipping label generation.
No XML file found for N249.jpeg, skipping label generation.
No XML file found for N180.jpeg, skipping label generation.
Dataset structure created with train and validation sets.


In [7]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 24.7 MB/s eta 0:00:00


### Creating the Dataset YAML File
This code snippet creates a YAML configuration file necessary for training the YOLO model:
- **YAML Content Definition**: Specifies paths to the training and validation image directories, the number of classes (`nc`), and the names of those classes. In this case, there is one class, 'number_plate'.
- **File Writing**: Opens a new file named `dataset.yaml` in write mode and saves the YAML content to this file.
- **Completion Message**: Prints a confirmation that the `dataset.yaml` file has been successfully created, which will be used by the YOLO model during training to understand the dataset structure.


In [ ]:
yaml_content = """train: /content/drive/MyDrive/dataset/images/train
val: /content/drive/MyDrive/dataset/images/val

nc: 1  # Number of classes
names: ['number_plate']  # List of class names
"""

with open('/content/drive/MyDrive/dataset/dataset.yaml', 'w') as file:
    file.write(yaml_content)

print("dataset.yaml file has been created.")

dataset.yaml file has been created.


### Training the YOLO Model
This code snippet initializes and trains a YOLO model using the Ultralytics library:
- **Model Loading**: Loads a pre-trained YOLOv8 model (`yolov8n.pt`), which serves as the starting point for training on the specific dataset.
- **Model Training**: Calls the `train` method, specifying the path to the dataset YAML file (`dataset.yaml`), the number of epochs for training (100), and the image size for training (640 pixels). This method will optimize the model's weights to detect number plates based on the provided training data.


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  

model.train(data='/content/drive/MyDrive/dataset/dataset.yaml', epochs=100, imgsz=640)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 107MB/s]


Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/dataset/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

100%|██████████| 755k/755k [00:00<00:00, 20.7MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all         46         45      0.978      0.975      0.992      0.697



100 epochs completed in 0.157 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.33it/s]


                   all         46         45      0.972      0.978      0.992      0.712
Speed: 0.2ms preprocess, 3.0ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a6341e909d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Saving the YOLO Model Weights
This code snippet saves the trained YOLO model's weights to a specified directory:
- **Directory Definition**: Sets the path for the directory where model weights will be saved (`models_dir`).
- **Directory Creation**: Uses `os.makedirs` to create the directory if it does not already exist, ensuring that the model can be saved without errors.
- **Model Saving**: Calls the `save` method on the model object to write the trained weights to a file named `my_yolov8_model.pt` in the specified directory.
- **Completion Message**: Prints a confirmation that the model weights have been successfully saved for future use or inference.


In [ ]:
models_dir = '/content/drive/MyDrive/models'

os.makedirs(models_dir, exist_ok=True)

model.save('/content/drive/MyDrive/models/my_yolov8_model.pt')

print("Model weights have been saved.")

Model weights have been saved.


### Running Inference on Test Images
This code snippet performs inference using the trained YOLO model on a set of test images:
- **Model Loading**: Loads the trained YOLO model from the specified path to perform inference.
- **Test Images Directory**: Specifies the directory containing the test images and an output directory for saving the results.
- **File Iteration**: Loops through each image file in the test directory, checking for common image file extensions.
- **Inference Execution**: Runs the model's `predict` method on each image, with results being saved to the output directory.
- **Image Processing**: If results are found:
  - Reads and converts the image color format from BGR to RGB for proper display.
  - Checks if any bounding boxes were detected; if not, it skips to the next image.
  - For each detected bounding box, it extracts the coordinates, confidence score, and class ID, then draws the bounding box and label on the image.
- **Image Display**: Uses `matplotlib` to display the processed image with bounding boxes and labels.
- **Completion Message**: Prints a confirmation that inference has been completed for all test images.


In [ ]:

model = YOLO('/content/drive/MyDrive/models/my_yolov8_model.pt')  

test_images_dir = '/content/drive/MyDrive/ANPR Test/ANPR Test'

output_dir = '/content/drive/MyDrive/ANPR_Test_Results'
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(test_images_dir):
    if filename.endswith(('.jpeg', '.jpg', '.png')):  
        image_path = os.path.join(test_images_dir, filename)

        results = model.predict(source=image_path, save=True, save_dir=output_dir)

        if results:
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

            for result in results:
                boxes = result.boxes
                if len(boxes) == 0:
                    print(f"No detections for {filename}.")
                    continue 

                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()  
                    confidence = box.conf[0].cpu().item()  
                    class_id = int(box.cls[0].cpu().item()) 

                    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
                    label = f'Class {class_id} {confidence:.2f}'
                    cv2.putText(img, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            plt.imshow(img)
            plt.title(f'Detections for {filename}')
            plt.axis('off')
            plt.show()
        else:
            print(f"No results for {filename}.")

print("Inference completed on ANPR Test images.")

Output hidden; open in https://colab.research.google.com to view.